## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()


C:\Users\neeha\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
#from google.colab import drive
#drive.mount('/gdrive')

In [4]:
import pandas as pd

In [5]:
data_prices = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [6]:
data_prices.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [7]:
data_prices.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [8]:
data_prices.describe()

,open,close,low,high,volume
count,851264.000000,851264.000000,851264.000000,851264.000000,8.512640e+05
mean,70.836986,70.857109,70.118414,71.543476,5.415113e+06
std,83.695876,83.689686,82.877294,84.465504,1.249468e+07
min,0.850000,0.860000,0.830000,0.880000,0.000000e+00
25%,33.840000,33.849998,33.480000,34.189999,1.221500e+06
50%,52.770000,52.799999,52.230000,53.310001,2.476250e+06
75%,79.879997,79.889999,79.110001,80.610001,5.222500e+06
max,1584.439941,1578.130005,1549.939941,1600.930054,8.596434e+08


### Drop columns `date` and  `symbol`

In [9]:
data_prices=data_prices[[ 'open', 'close', 'low', 'high', 'volume']]

In [10]:
data_prices.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [11]:
data_prices['volume']=data_prices['volume']/1000000

In [12]:
data_prices=data_prices.head(1000)

In [13]:
data_prices.shape

(1000, 5)

### Divide the data into train and test sets

In [86]:
from sklearn.model_selection import train_test_split

In [87]:
y=data_prices['volume']
X=data_prices[[ 'open', 'close', 'low', 'high']]

In [88]:
X_arr=np.array(X).astype(np.float32)
y_arr=np.array(y).astype(np.float32)

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X_arr, y_arr, test_size=0.2, random_state=42)

#### Convert Training and Test Data to numpy float32 arrays


In [90]:
#X_train_arr=np.array(X_train).astype(np.float32)
X_test.dtype

dtype('float32')

In [91]:
#X_test_arr=np.array(X_test).astype(np.float32)
X_train.dtype

dtype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [92]:
from sklearn.preprocessing import Normalizer

nm=Normalizer()

In [93]:
X_train=nm.fit_transform(X_train)
X_test=nm.transform(X_test)

In [94]:
X_train.shape

(800, 4)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [170]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [171]:
def predict_model(X_train,W,b):#X_train
    return tf.add(tf.matmul(X_train,W),b,name='output')

3.Loss (Cost) Function [Mean square error]

In [172]:
def loss_func(y,y_pred):#y_train, y_ for prediction
    return tf.reduce_mean(tf.square(y-y_pred),name='mse')

In [173]:
def train(X, y, W, b):#X_train & y_train
    with tf.GradientTape() as t:
        t.watch([W,b])
        y_pred=predict_model(X,W,b)
        loss=loss_func(y,y_pred) 
    DW,Db =t.gradient(loss,[W,b])
    W=W-DW*0.03
    b=b-Db*0.03
    return W,b,loss

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [175]:
W,b,l = train(X_train, y_train, W, b) #Training by passing X_train and y_train

In [176]:
l

<tf.Tensor: id=12060, shape=(), dtype=float32, numpy=229.31682>

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [177]:
epochs =100

In [179]:
for epoch in range(epochs):
    y_pred=predict_model(X_train,W,b)
    loss=loss_func(y_train,y_pred)
    W,b,l =train(X_train, y_train, W, b)
    
    print(epoch,loss)

0 tf.Tensor(223.2096, shape=(), dtype=float32)
1 tf.Tensor(218.48, shape=(), dtype=float32)
2 tf.Tensor(214.81747, shape=(), dtype=float32)
3 tf.Tensor(211.9812, shape=(), dtype=float32)
4 tf.Tensor(209.78473, shape=(), dtype=float32)
5 tf.Tensor(208.0838, shape=(), dtype=float32)
6 tf.Tensor(206.76657, shape=(), dtype=float32)
7 tf.Tensor(205.74644, shape=(), dtype=float32)
8 tf.Tensor(204.9565, shape=(), dtype=float32)
9 tf.Tensor(204.34483, shape=(), dtype=float32)
10 tf.Tensor(203.87103, shape=(), dtype=float32)
11 tf.Tensor(203.50421, shape=(), dtype=float32)
12 tf.Tensor(203.22014, shape=(), dtype=float32)
13 tf.Tensor(203.00009, shape=(), dtype=float32)
14 tf.Tensor(202.82977, shape=(), dtype=float32)
15 tf.Tensor(202.6978, shape=(), dtype=float32)
16 tf.Tensor(202.59567, shape=(), dtype=float32)
17 tf.Tensor(202.51657, shape=(), dtype=float32)
18 tf.Tensor(202.4552, shape=(), dtype=float32)
19 tf.Tensor(202.40773, shape=(), dtype=float32)
20 tf.Tensor(202.3711, shape=(), dtype=

### Get the shapes and values of W and b

In [166]:
W.shape

TensorShape([Dimension(4), Dimension(1)])

In [167]:
b.shape

TensorShape([Dimension(1)])

### Model Prediction on 1st Examples in Test Dataset

In [180]:
y_test_pred=predict_model(X_test[0],W,b)

In [184]:
y_test_pred[0]

<tf.Tensor: id=17310, shape=(1,), dtype=float32, numpy=array([5.202331], dtype=float32)>

In [185]:
y_test[0]

2.6424

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

In [78]:
import pandas as pd

### Load the given Iris data using pandas (Iris.csv)

In [79]:
data_Iris = pd.read_csv('Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [80]:
data_Iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [81]:
data_Iris=pd.get_dummies(data_Iris)

In [82]:
data_Iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [83]:
X = data_Iris.iloc[:,1:5].values
Y = data_Iris.iloc[:,5:8].values

In [84]:
Y.shape

(150, 3)

### Divide the dataset into Training and test (70:30)

In [85]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=42) 

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [86]:
import tensorflow as tf
import pandas as pd
import numpy as np

#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
#Input Layer
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically

#Hidden layers
model.add(tf.keras.layers.Dense(4,activation='tanh'))
model.add(tf.keras.layers.Dense(4,activation='tanh'))

#outputlayer
model.add(tf.keras.layers.Dense(3,activation='softmax'))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='categorical_crossentropy') # we can use mae also (Mean absolute error)

### Model Training 

In [87]:
model.fit(X_train, y_train, epochs=100,validation_data=[X_test,y_test])

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 1s 8ms/sample - loss: 0.9042 - val_loss: 1.0416
Epoch 2/100
105/105 [==============================] - 0s 160us/sample - loss: 0.8924 - val_loss: 1.0346
Epoch 3/100
105/105 [==============================] - 0s 164us/sample - loss: 0.8776 - val_loss: 1.0285
Epoch 4/100
105/105 [==============================] - 0s 163us/sample - loss: 0.8745 - val_loss: 1.0217
Epoch 5/100
105/105 [==============================] - 0s 186us/sample - loss: 0.8639 - val_loss: 1.0147
Epoch 6/100
105/105 [==============================] - 0s 176us/sample - loss: 0.8476 - val_loss: 1.0079
Epoch 7/100
105/105 [==============================] - 0s 230us/sample - loss: 0.8477 - val_loss: 1.0012
Epoch 8/100
105/105 [==============================] - 0s 124us/sample - loss: 0.8255 - val_loss: 0.9949
Epoch 9/100
105/105 [==============================] - 0s 115us/sample - loss: 0.8233 - val_loss: 0.9881
Epoch 10/100

105/105 [==============================] - 0s 93us/sample - loss: 0.4407 - val_loss: 0.4404
Epoch 79/100
105/105 [==============================] - 0s 76us/sample - loss: 0.4443 - val_loss: 0.4342
Epoch 80/100
105/105 [==============================] - 0s 102us/sample - loss: 0.4378 - val_loss: 0.4299
Epoch 81/100
105/105 [==============================] - 0s 103us/sample - loss: 0.4241 - val_loss: 0.4262
Epoch 82/100
105/105 [==============================] - 0s 53us/sample - loss: 0.4584 - val_loss: 0.4226
Epoch 83/100
105/105 [==============================] - 0s 119us/sample - loss: 0.4288 - val_loss: 0.4192
Epoch 84/100
105/105 [==============================] - 0s 107us/sample - loss: 0.4150 - val_loss: 0.4156
Epoch 85/100
105/105 [==============================] - 0s 83us/sample - loss: 0.4416 - val_loss: 0.4098
Epoch 86/100
105/105 [==============================] - 0s 104us/sample - loss: 0.4158 - val_loss: 0.4068
Epoch 87/100
105/105 [==============================] - 0s 102u

### Model Prediction

In [88]:
model.predict(X_test)

array([[0.06757179, 0.69205844, 0.2403697 ],
       [0.8077095 , 0.16089395, 0.03139647],
       [0.03253098, 0.18811189, 0.77935714],
       [0.06708154, 0.44726694, 0.48565152],
       [0.06002674, 0.68524146, 0.25473186],
       [0.7988484 , 0.17037125, 0.03078038],
       [0.13639174, 0.6332974 , 0.2303109 ],
       [0.03420854, 0.16264848, 0.80314296],
       [0.05457136, 0.7769833 , 0.16844533],
       [0.07711798, 0.75691754, 0.16596448],
       [0.03824357, 0.17108506, 0.79067135],
       [0.7992838 , 0.1724534 , 0.02826289],
       [0.80550385, 0.16551974, 0.02897646],
       [0.8009627 , 0.17080995, 0.02822729],
       [0.81873673, 0.15135206, 0.02991119],
       [0.06814331, 0.23713121, 0.6947254 ],
       [0.03127911, 0.15625574, 0.81246513],
       [0.0711083 , 0.7912048 , 0.13768692],
       [0.07834115, 0.59953356, 0.32212523],
       [0.03217573, 0.16831236, 0.79951185],
       [0.804382  , 0.16745713, 0.02816084],
       [0.04355762, 0.21404305, 0.7423993 ],
       [0.

In [89]:
y_pred = model.predict_classes(X_test)

In [90]:
y_pred

array([1, 0, 2, 2, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 2, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 0,
       0], dtype=int64)

In [91]:
y_pred[0:5]

array([1, 0, 2, 2, 1], dtype=int64)

In [92]:
y_test.argmax(axis=1)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0], dtype=int64)

In [93]:
y_test_val=y_test.argmax(axis=1)

In [94]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print(metrics.accuracy_score(y_test_val,y_pred))

0.8888888888888888


### Save the Model

In [95]:
model.save('R6Lab_TF_&_Keras.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?